In [2]:
import numpy as np
import polars as pl
import matplotlib as plt

In [3]:
df = pl.read_csv("../data/selected_features_inflation.csv", infer_schema_length=10000, dtypes={"ANDENOx": pl.Float64})

display(df.head())

print("\nDataframe info:")
print(f"Shape: {df.shape}")
print(f"Column names: {df.columns}")
print(f"Data types:\n{df.dtypes}")

/var/folders/jb/ckb_b9f94m5dvhsy6d9bwn800000gn/T/ipykernel_48852/3781630620.py:1: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  df = pl.read_csv("../data/selected_features_inflation.csv", infer_schema_length=10000, dtypes={"ANDENOx": pl.Float64})


sasdate,CPIAUCSL_YOY,CUSR0000SA0L5_YOY,ACOGNO_YOY,TOTRESNS_YOY,CUSR0000SAD_YOY,AMDMUOx_YOY,S&P PE ratio_MOM,HWIURATIO,OILPRICEx_YOY,USCONS_YOY,CPIMEDSL_YOY,WPSFD49207_YOY,TOTRESNS_MOM,IPNMAT_YOY,S&P 500_MOM,CES1021000001,UEMP27OV_YOY,WPSID61_YOY,USTRADE,CES3000000008_YOY,RPI_YOY,CUSR0000SAS_YOY,GS10,S&P PE ratio_YOY,IPB51222S,EXUSUKx,DTCTHFNM_YOY,WPSID61_MOM,COMPAPFFx,ANDENOx,CES0600000007,CLAIMSx,IPDCONGD_YOY,IPDCONGD,OILPRICEx_MOM,HOUSTNE,…,DSERRG3M086SBEA_YOY,EXJPUSx_YOY,ACOGNO_MOM,EXJPUSx_MOM,UEMP5TO14_MOM,EXSZUSx_MOM,BUSLOANS_YOY,CES1021000001_MOM,CLF16OV_MOM,DTCTHFNM_MOM,EXCAUSx_MOM,UEMPLT5_MOM,CMRMTSPLx_MOM,CLF16OV_YOY,CPIMEDSL_MOM,PERMITNE,TB6SMFFM,WPSID62_MOM,AMDMNOx_YOY,M1SL_MOM,IPBUSEQ_MOM,EXUSUKx_MOM,CES2000000008_MOM,PPICMM_YOY,IPCONGD_MOM,PPICMM_MOM,UEMP15T26_YOY,IPNCONGD_YOY,USGOVT_MOM,HOUSTW,NDMANEMP_MOM,IPDCONGD_MOM,AMDMNOx_MOM,IPNMAT_MOM,DDURRG3M086SBEA_MOM,DDURRG3M086SBEA_YOY,CPIAPPSL_MOM
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64
"""1959-01-01""",null,null,null,null,null,null,null,0.333579,null,null,null,null,null,null,null,713.0,null,null,5350.3,null,null,null,4.02,null,19.9173,2.8065,null,null,0.82,null,39.8,291078,null,18.9498,null,350,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.61,null,null,null,null,null,null,null,null,null,null,null,null,350,null,null,null,null,null,null,null
"""1959-02-01""",null,null,null,null,null,null,-0.149749,0.358386,null,null,null,null,-1.587302,null,-1.528227,704.2,null,null,5381.3,null,null,null,3.96,null,19.8631,2.8093,null,0.326797,0.83,null,39.7,282958,null,19.0492,0.0,346,…,null,null,null,0.0,-0.427716,0.025509,null,-1.234222,-0.422456,0.0,0.796195,-1.270648,0.736297,null,0.473934,null,0.7,-0.632911,null,0.359971,1.338499,0.099768,0.408163,null,0.733585,0.0,null,null,0.135719,344,0.287632,0.524544,4.646258,1.751079,0.058252,null,-0.223214
"""1959-03-01""",null,null,null,null,null,null,3.155764,0.400947,null,null,null,null,-1.075269,null,2.537886,704.1,null,null,5431.6,null,null,null,3.99,null,19.9986,2.8127,null,0.0,0.55,null,40.0,260346,null,19.4223,-1.0,330,…,null,null,null,0.0,-6.099656,0.220249,null,-0.014201,0.619374,0.414702,-0.512926,-6.113256,-0.336805,null,0.471698,null,0.33,0.318471,null,0.215208,1.08067,0.121027,-0.406504,null,0.0,1.230769,null,null,0.197141,374,0.286807,1.958612,2.241358,1.502019,0.125877,null,0.0
"""1959-04-01""",null,null,null,null,null,null,1.440165,0.444973,null,null,null,null,1.630435,null,1.673789,705.2,null,null,5395.4,null,null,null,4.12,null,20.0799,2.8165,null,0.0,0.46,null,40.2,246413,null,19.5466,0.0,275,…,null,null,null,0.0,-14.547118,-0.004627,null,0.156228,0.398131,1.093206,-0.639307,2.398903,2.011502,null,0.0,null,0.31,0.634921,null,0.0,2.613397,0.135102,0.816327,null,1.577494,-0.607903,null,null,0.122971,388,0.222434,0.639986,1.105071,3.09727,0.100575,null,0.223714
"""1959-05-01""",null,null,null,null,null,null,1.147522,0.475711,null,null,null,null,-0.534759,null,1.50613,710.0,null,null,5456.3,null,null,null,4.31,null,20.3779,2.8145,null,0.651466,0.66,null,40.3,246952,null,19.9445,0.0,260,…,null,null,null,0.0,7.601713,0.004627,null,0.680658,-0.23559,1.297661,-0.062267,-1.004016,0.686227,null,0.469484,null,0.43,-0.630915,null,0.71582,3.240249,-0.07101,0.404858,null,0.47775,0.611621,null,null,0.135102,396,0.301205,2.035648,-4.212454,0.898387,0.16641,null,0.223214



Dataframe info:
Shape: (793, 103)
Column names: ['sasdate', 'CPIAUCSL_YOY', 'CUSR0000SA0L5_YOY', 'ACOGNO_YOY', 'TOTRESNS_YOY', 'CUSR0000SAD_YOY', 'AMDMUOx_YOY', 'S&P PE ratio_MOM', 'HWIURATIO', 'OILPRICEx_YOY', 'USCONS_YOY', 'CPIMEDSL_YOY', 'WPSFD49207_YOY', 'TOTRESNS_MOM', 'IPNMAT_YOY', 'S&P 500_MOM', 'CES1021000001', 'UEMP27OV_YOY', 'WPSID61_YOY', 'USTRADE', 'CES3000000008_YOY', 'RPI_YOY', 'CUSR0000SAS_YOY', 'GS10', 'S&P PE ratio_YOY', 'IPB51222S', 'EXUSUKx', 'DTCTHFNM_YOY', 'WPSID61_MOM', 'COMPAPFFx', 'ANDENOx', 'CES0600000007', 'CLAIMSx', 'IPDCONGD_YOY', 'IPDCONGD', 'OILPRICEx_MOM', 'HOUSTNE', 'REALLN_MOM', 'BOGMBASE_MOM', 'DTCOLNVHFNM_YOY', 'NDMANEMP_YOY', 'TWEXAFEGSMTHx_YOY', 'ANDENOx_MOM', 'CPIAPPSL_YOY', 'DSERRG3M086SBEA_MOM', 'IPFUELS_MOM', 'RETAILx_MOM', 'W875RX1_MOM', 'CUSR0000SAS_MOM', 'CUSR0000SA0L5_MOM', 'CUSR0000SAD', 'BUSINVx_MOM', 'UEMPLT5_YOY', 'INVEST_MOM', 'EXSZUSx', 'IPMAT_YOY', 'UMCSENTx', 'BOGMBASE_YOY', 'UEMP15OV_MOM', 'RETAILx_YOY', 'USGOVT_YOY', 'CES300000000

In [4]:
import statsmodels as sm
import statsmodels.api as sm
from statsmodels.formula.api import ols
import datetime

target = 'CPIAUCSL_YOY'
predictor = 'CUSR0000SA0L5_YOY'

# Create a lagged version of the predictor to avoid lookahead bias
df_with_lag = df.with_columns(
    pl.col(predictor).shift(1).alias(f"{predictor}_lag1")
)

# Filter data to include only post-2008 observations
post_2008_df = df_with_lag.filter(
    pl.col("sasdate").str.strptime(pl.Date, "%Y-%m-%d") >= datetime.date(2008, 1, 1)
)



train_df = post_2008_df.select(["sasdate", target, f"{predictor}_lag1"])


train_df = train_df.drop_nulls()

formula = f"{target} ~ {predictor}_lag1"
model = ols(formula, data=train_df.to_pandas()).fit()

print(f"\nOLS Regression Results for {target} ~ {predictor} (lagged by 1) - Post 2008 Data")
print(model.summary())


OLS Regression Results for CPIAUCSL_YOY ~ CUSR0000SA0L5_YOY (lagged by 1) - Post 2008 Data
                            OLS Regression Results                            
Dep. Variable:           CPIAUCSL_YOY   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.950
Method:                 Least Squares   F-statistic:                     3837.
Date:                Wed, 02 Apr 2025   Prob (F-statistic):          8.25e-134
Time:                        13:17:37   Log-Likelihood:                -131.05
No. Observations:                 205   AIC:                             266.1
Df Residuals:                     203   BIC:                             272.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

In [5]:
# Calculate Mean Squared Error (MSE) for the OLS model, but only for dates between 1970 and 2005
from sklearn.metrics import mean_squared_error
import datetime

filtered_df = df_with_lag.with_columns(
    pl.col("sasdate").str.strptime(pl.Date, "%Y-%m-%d").alias("date")
).filter(
    (pl.col("date") >= datetime.date(1970, 1, 1)) & 
    (pl.col("date") <= datetime.date(2005, 12, 31))
)

y_true_filtered = filtered_df[target]
y_pred_filtered = model.predict(filtered_df.to_pandas())

mse_filtered = mean_squared_error(y_true_filtered, y_pred_filtered)
rmse_filtered = np.sqrt(mse_filtered)
r2_filtered = model.rsquared

print(f"\nResults for dates between 1970-2005:")
print(f"Mean Squared Error (MSE): {mse_filtered:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse_filtered:.6f}")
print(f"R-squared: {r2_filtered:.6f}")



Results for dates between 1970-2005:
Mean Squared Error (MSE): 0.292938
Root Mean Squared Error (RMSE): 0.541237
R-squared: 0.949759


In [6]:
plt.figure(figsize=(10, 6))
plt.scatter(filtered_df['CUSR0000SA0L5_YOY_lag1'], filtered_df[target], alpha=0.5)
plt.plot(filtered_df['CUSR0000SA0L5_YOY_lag1'], y_pred_filtered, color='red', linewidth=2)
plt.xlabel(predictor)
plt.ylabel(target)
plt.title(f'OLS Regression: {target} ~ {predictor} (1970-2005)')
plt.grid(True, alpha=0.3)
plt.show()

NameError: name 'plt' is not defined

In [1]:
# Assuming you have your test data in filtered_df
# and your model is already trained as 'model'

# The predictions were already created (y_pred_filtered)
# Calculate residuals
residuals = filtered_df[target] - y_pred_filtered

plt.figure(figsize=(10, 6))

plt.scatter(y_pred_filtered, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='-', linewidth=2)
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot (Test Sample)")
plt.grid(True, alpha=0.3)

z = np.polyfit(y_pred_filtered, residuals, 1)
p = np.poly1d(z)
plt.plot(y_pred_filtered, p(y_pred_filtered), "r--", alpha=0.8)

plt.tight_layout()
plt.show()

NameError: name 'filtered_df' is not defined